# Exercise 3

## Helper functions

In [1]:
import numpy as np
import cv2 
import os
import scipy
import matplotlib.pyplot as plt
import matplotlib

def load_im(path : str) -> np.ndarray:
    
    im = cv2.imread(path)[:, :, ::-1]
    im = im.astype(np.float64) / 255
    
    return im

def pi(points : np.ndarray) -> np.ndarray:
    """
        Converts from homogeneous to inhomogeneous coordinates
    """
    p = points[:-1]/points[-1]
    
    return p


def piInv(points : np.ndarray) -> np.ndarray:
    """
        Converts from inhomogeneous to homogeneous coordinates
    """
    
    # Gets the amount of points by using shape
    _, num_points = points.shape
    
    # Stacks the scale s at the bottom of the matrix
    ph = np.vstack((points, np.ones(num_points)))
    
    return ph


def projectPoints(K, Rt, Q):
    
    Q_hom = piInv(Q)
    points = K @ Rt @ Q_hom
    points_inhom = pi(points)
    
    return points_inhom

## Initial setup
Its propaply good to remember that:
$$
    \begin{align*}
        \pmb{p}_h &= \pmb{K} \pmb{P}_{cam}
        \\
        &=  \pmb{K} \left[ \pmb{R} \pmb{t} \right] \pmb{P}_h
    \end{align*}
$$

Where:
$$
    \begin{equation*}
        \mathcal{P} = \pmb{K} \left[ \pmb{R} \pmb{t} \right]
    \end{equation*}
$$

In [2]:

K = np.array(
    [[1000, 0, 300],
     [0, 1000, 200],
     [0, 0, 1]]
)

R1 = np.diag(np.ones(3))
t1 = np.zeros((3,1))
Rt1 = np.concatenate((R1, t1), axis=1)

R2 = scipy.spatial.transform.Rotation.from_euler("xyz", [0.7, -0.5, 0.8]).as_matrix()
t2 = np.array([0.2, 2, 1]).reshape(-1, 1)
Rt2 = np.concatenate((R2, t2), axis=1)


## Ex 3.1
Consider the 3D point:

$$
    \begin{bmatrix}
        1 \\
        0.5 \\
        4 \\
        1
    \end{bmatrix}
$$

and find the projections in `Cam1` and `Cam2`, respectively, points $\pmb{q}_1$ and $\pmb{q}_2$.

*Okay... and now I can't be bothered to write the exercise description anymore*

### Response

We use the formulas:
$$
    \begin{align*}
        \pmb{q} &= \mathcal{P} \pmb{p} \\
        \pmb{p} &= \mathcal{P}^{-1} \pmb{q}
    \end{align*}
$$
from week 3, slide 14.

In [3]:
Q = np.array([1, 0.5, 4, 1]).reshape(-1, 1)

q1 = K @ Rt1 @ Q
q2 = K @ Rt2 @ Q

print(f"q1: \n{q1}\n")
print(f"q2: \n{q2}")
      



q1: 
[[2200.]
 [1300.]
 [   4.]]

q2: 
[[2590.22714938]
 [ 827.08784113]
 [   4.44695131]]


## Ex 3.2

We now need to implement a function called `CrossOp`.


### Response

In [4]:
def CrossOp(p : np.ndarray) -> np.ndarray:
    """
        One of Them weird functions. It takes in a 3D vector and then returns
        some gnarly matrix.
    """
    p = p.flatten()
    if p.size != 3:
        raise Exception("Invalid input, vector must be exactly 3D.")
    
    x, y, z = p
    px = np.array(
        [[0, -z, y],
         [z, 0, -x],
         [-y, x, 0]]
    )
    
    return px

#### Testing
this dumb-ass function

In [5]:
p1 = np.array([2, 3, 4])
p2 = np.array([6, 4, 1])

p1x = CrossOp(p1)

p1xp2 = np.cross(p1, p2)

print(f"Cross product between q1 and q2: \n{p1xp2}\nCrossOp between q1 and q2: \n{p1x @ p2}\nTheir difference: \n{p1xp2 - p1x @ p2}")

Cross product between q1 and q2: 
[-13  22 -10]
CrossOp between q1 and q2: 
[-13  22 -10]
Their difference: 
[0 0 0]


Pretty numbers give pretty results <3

## Ex 3.3
We need to compute fundamental matrix $\pmb{F}$ of the two cameras.

### Response
We use this formula from week 3, slide 20:

For:
$$
    \begin{equation*}
        \pmb{p} = \mathcal{P}^{-1} \pmb{q}
    \end{equation*}
$$

We have:
$$
    \begin{align*}
        \pmb{p}_2^T \pmb{E} \pmb{p}_1 &= 0
        \\
        (\pmb{K}^{-1}_2 \pmb{q}_2)^T \pmb{E} (\pmb{K}^{-1}_1 \pmb{q}_1) &= 0
        \\
        \pmb{q}_2^T \pmb{K}^{-T}_2 \pmb{E} \pmb{K}^{-1}_1 \pmb{q}_1 &= 0
    \end{align*}
$$

Where:
$$
    \begin{equation*}
        \pmb{F} = \pmb{K}^{-T}_2 \pmb{E} \pmb{K}^{-1}_1
    \end{equation*}
$$

But to compute this we need to use the *Essential matrix* $\pmb{E}$ found on week 3, slide 17:
$$
    \begin{align*}
        \pmb{n} &= \pmb{t} \times (\pmb{R} \pmb{p}_1 + \pmb{t})
        \\
        &= \pmb{t} \times (\pmb{R} \pmb{p}_1)
        \\
        &= \left[\pmb{t} \right]_{\times} \pmb{R} \pmb{p}_1.
    \end{align*}
$$

Where:
$$
    \begin{equation*}
        \pmb{E} = \left[\pmb{t} \right]_{\times} \pmb{R}.
    \end{equation*}
$$


As Alex says: because our reference frame is `Cam1`, we need to use the $\pmb{R}_2$ and $\pmb{t}_2$ to create the epipolar plane (the plane between the two cameras).

In [6]:
t2x = CrossOp(t2)

E = t2x @ R2

Kinv = np.linalg.inv(K)

F = Kinv.T @ E @ Kinv

print(f"The fundamental matrix: \n{F}")

The fundamental matrix: 
[[ 3.29311881e-07  8.19396327e-07  1.79162592e-03]
 [ 5.15532551e-07 -8.76915984e-07  9.31426656e-05]
 [-1.29882755e-03  1.51951700e-03 -1.10072682e+00]]


## Ex 3.4
What is the epipolar line $\pmb{l}$ of $\pmb{q}_1$ in camera two?

### Reponse
We can aparently use week 3, slide 19.
It says that

For:
$$
    \begin{equation*}
        \pmb{p} = \mathcal{P}^{-1} \pmb{q}
    \end{equation*}
$$

Are $\pmb{p}_1$ and $\pmb{p}_2$ in homogenous coordinates?

There are two interpretations:
*  $\pmb{p}_1$ and $\pmb{p}_2$ are 3D points and $\pmb{n}$ is a vector in 3D.
* $\pmb{p}_1$ and $\pmb{p}_2$ are 2D points and $\pmb{n} = \pmb{E} \pmb{p}_1$ is the epipolar line, both are in homogenous coordinates.

In [7]:
n = E @ q1

print(f"Is this the epipolar line? \n{n}\n")

# No! Cause we need to use F, since we are in homogenous coordinates
print(f"This is the epipolar line! \n{F @ q1}")


Is this the epipolar line? 
[[1797.91855829]
 [  -5.5294899 ]
 [-348.52473187]]

This is the epipolar line! 
[[ 8.95620504e-03]
 [ 3.66751496e-04]
 [-5.28495581e+00]]


## Ex 3.5
Is $\pmb{q}_2$ located on the epipolar line from **Ex. 3.4**?

### Response
We use that if the vectors are orthogonal then we now the point lies on the line. 
This makes intuitive sence in that two vectors that are orthogonal must intersect.
We know that the dot product between two orthogonal vectors is zero.

In [8]:
epi_line = F @ q1

print(f"Is this close to zero? \n{q2.T @ epi_line}")

Is this close to zero? 
[[7.10542736e-15]]


Then we can argue that the epipolar line is constructed in such a manner that it will always be orthogonal to the vector drawn from the camera center to the corresponding point on the camera.

For further clarification inspect the figures of week 3, slides 8-12.

## Ex 3.6

A lot of text!

Let $\pmb{Q}$ and $\tilde{\pmb{Q}}$ denote the *same* 3D point in the world space $\pmb{Q}$, and in the frame of camera one $\tilde{\pmb{Q}}$.

In other words we have the relation:
$$
    \begin{equation*}
        \tilde{\pmb{Q}} =
        \begin{bmatrix}
            \pmb{R}_1, \pmb{t} \\
            \pmb{0}, 1
        \end{bmatrix}
        \pmb{Q}
    \end{equation*}
$$

Then show analytically that:
$$
    \begin{equation*}
        \pmb{Q} =
        \begin{bmatrix}
            \pmb{R}_1^T, -\pmb{R}_1^T \pmb{t}_1 \\
            \pmb{0}, 1
        \end{bmatrix}
        \tilde{\pmb{Q}}
    \end{equation*}
$$

### Response

## Ex 3.7

Show that the projection can work only in the coordinate system of camera one, by showing we can project points with:
$$
    \begin{equation*}
        \pmb{q}_1 = \pmb{K} \left[ \pmb{I} \pmb{0} \right] \tilde{\pmb{Q}}, \quad \text{and} \quad \pmb{q}_2 = \pmb{K} \left[ \tilde{\pmb{R}}_2 \tilde{\pmb{t}}_2 \right] \tilde{\pmb{Q}}
    \end{equation*}
$$

where:
$$
    \begin{equation*}
        \tilde{\pmb{R}}_2 = \pmb{R}_2 \pmb{R}_1^, \quad \text{and} \quad \tilde{\pmb{t}}_2 = \pmb{t}_2 - \pmb{R}_2 \pmb{R}_1^T \pmb{t}_1.
    \end{equation*}
$$


### Response

I just show that it works with the code

First a function to get that weird matrix

In [9]:
def weirdMat(R, T):
    m = R.shape[1]
    bla = np.concatenate((R, np.zeros((1, m))), axis=0)
    blaBla = np.concatenate((t1, np.array(1).reshape(-1,1)), axis=0)
    weird_mat = np.concatenate((bla, blaBla), axis=1)
    
    return weird_mat
    

In [10]:
weird_mat1 = weirdMat(R1, t1)
weird_mat2 = weirdMat(R2, t2)

print("Weird matrix 1:")
print(weird_mat1)

print("\nWeird matrix 2:")
print(weird_mat2)

Q_tilde1 = weird_mat1 @ Q
Q_tilde2 = weird_mat2 @ Q

print("\nQ tilde 1:")
print(Q_tilde1)

print("\nQ tilde 2:")
print(Q_tilde2)



Weird matrix 1:
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]

Weird matrix 2:
[[ 0.61141766 -0.76384514  0.20666167  0.        ]
 [ 0.6295392   0.31131209 -0.71187442  0.        ]
 [ 0.47942554  0.56535421  0.67121217  0.        ]
 [ 0.          0.          0.          1.        ]]

Q tilde 1:
[[1. ]
 [0.5]
 [4. ]
 [1. ]]

Q tilde 2:
[[ 1.05614176]
 [-2.06230242]
 [ 3.44695131]
 [ 1.        ]]


#### First showing $\pmb{q}_1$

In [11]:
q1_approx = K @ np.concatenate((np.eye(K.shape[1]), np.zeros((K.shape[0], 1))), axis=1) @ Q_tilde2

print(f"q1: \n{q1}\n\nq1 approximated: \n{q1_approx}")

q1: 
[[2200.]
 [1300.]
 [   4.]]

q1 approximated: 
[[ 2090.22714938]
 [-1372.91215887]
 [    3.44695131]]


#### Secondly showing $\pmb{q}_2$

In [12]:
R2_tilde = R2 @ R1.T
t2_tilde = t2 - R2 @ R1.T @ t1
q2_approx = K @ np.concatenate((R2_tilde, t2_tilde), axis = 1) @ Q_tilde2


print(f"q2: \n{q2}\n\nq2 approximated: \n{q2_approx}")

q2: 
[[2590.22714938]
 [ 827.08784113]
 [   4.44695131]]

q2 approximated: 
[[3.92958980e+03]
 [9.98756356e+01]
 [2.65404563e+00]]


*Soooo.. that worked horribly*

## Ex. 3.8
load a file and compute the fundamental matrix between camera 1 and camer 2.

### Response

#### Loading the shabang

In [13]:
DATA_PATH = "./data/TwoImageData.npy"
stuff = np.load(DATA_PATH, allow_pickle=True).item()

#print(stuff.keys())

im1 = stuff["im1"]
im2 = stuff["im2"]
R1 = stuff["R1"]
R2 = stuff["R2"]
t1 = stuff["t1"]
t2 = stuff["t2"]
K = stuff["K"]

print(f"R1: \n{R1} \n\nR2: \n{R2} \n\nt1: \n{t1} \n\nt2: \n{t2}")


R1: 
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]] 

R2: 
[[ 9.97063888e-01  1.21564307e-04  7.65740684e-02]
 [-1.37753401e-04  9.99999969e-01  2.06135565e-04]
 [-7.65740409e-02 -2.16078666e-04  9.97063874e-01]] 

t1: 
[[0.]
 [0.]
 [0.]] 

t2: 
[[-0.04594547]
 [-0.00017171]
 [ 0.00157051]]


#### Doing the math

In [14]:
t2x = CrossOp(t2)

E = t2x @ R2

Kinv = np.linalg.inv(K)

F = Kinv.T @ E @ Kinv

print(f"The fundamental matrix: \n{F}")

The fundamental matrix: 
[[ 6.67972386e-12 -7.85049967e-10  1.17921973e-07]
 [-9.75936980e-10 -4.86806510e-12  3.28699196e-05]
 [ 4.23506610e-07 -3.21704080e-05 -2.12002228e-04]]


## Ex. 3.9
Write code such that you can click on a point in an image and display its corresponding epipolar line in image two.



### Response

#### Helper functions

In [15]:
def in_frame(l, l_im, shape):
    """
        I think this checks whether the line is within the image
    """
    q = np.cross(l.flatten(), l_im)
    q = q[:2]/q[2]
    if all(q >= 0) and all(q+1 <= shape[1::-1]):
        return q
    
def DrawLine(l, shape):
    """
        Checks where the line intersects the four sides of the image
        and finds the two intersections that are within the frame
    """
    lines = [[1, 0, 0], [0, 1, 0], [1, 0, 1-shape[1]], [0, 1, 1-shape[0]]]
    P = [in_frame(l, l_im, shape) for l_im in lines if in_frame(l, l_im, shape) is not None]
    plt.plot(*np.array(P).T)
    

#### Actual code

In [16]:
%matplotlib qt

plt.figure(figsize=(16, 9))
plt.title("Image 1 naked")
plt.axis("off")
plt.subplot(1, 2, 1)
plt.imshow(im1, cmap="gray")

point = np.asarray(plt.ginput(1)).T
#print(point)

point_hom = piInv(point)
#print(point_hom)

epi_line = F @ point_hom

plt.subplot(1, 2, 2)
plt.title("Image 2 with epipolar line")
plt.axis("off")
plt.imshow(im2, cmap="gray")
DrawLine(epi_line, im2.shape)

plt.show()






## Ex. 3.10
Get epipolar line from image 2 to image 1.

### Response
The code

#### Helper funcs

In [17]:
def computeFundamentalMatrix(K1, K2, R2, t2):
    """
        Computing the fundamental matrix between two camera matrices K1 & K2.
    """
    t2x = CrossOp(t2)

    E = t2x @ R2

    K1inv = np.linalg.inv(K1)
    K2inv = np.linalg.inv(K2)

    F = K1inv.T @ E @ K2inv
    
    return F
    
    

Using equation (19) from exercises 3

In [20]:
%matplotlib qt
SECONDS_TO_TIMEOUT = 10

plt.figure(figsize=(16, 9))
plt.title("Image 1 naked")
plt.axis("off")
plt.subplot(1, 2, 1)
plt.imshow(im1, cmap="gray")

point = np.asarray(plt.ginput(1), timeout = SECONDS_TO_TIMEOUT).T

# Using that equation (19) from week 3.
R2t = R2 @ R1.T
t2t = t2 - R2t @ t1

F = computeFundamentalMatrix(K, K, R2t, t2t)

point_hom = piInv(point)
#print(point_hom)

epi_line = F @ point_hom

plt.subplot(1, 2, 2)
plt.title("Image 2 with epipolar line")
plt.axis("off")
plt.imshow(im2, cmap="gray")
DrawLine(epi_line, im2.shape)

plt.show()

TypeError: asarray() got an unexpected keyword argument 'timeout'

## Ex. 3.11 - Triangulation


### Response
For the code look at slide 28 from week 3.

In [ ]:
def triangulate(q_thicc : list, P_thicc : list):
    """
        Should take in:
            A list of n pixel-coordinates: [q1, q2, ..., qn]
            
            A list of n projection matrices: [P1, P2, ..., Pn]
        
        And return:
            The triangulation of the 3D point by utilizing the linear algorithm.
    """
    
    n = len(P_thicc)
    m = P_thicc[0].shape[1]
    
    B = np.zeros((2*n, m))
    
    for i in range(n):
        Pi = P_thicc[i]
        x, y = q_thicc[i]
        x, y = x.item(), y.item()   # <-- apparently there could be some issues with indexing of arrays
        
        B[i*2] = Pi[2] * x - Pi[0]
        B[i * 2 + 1] = Pi[2] * y - Pi[1]
        
    u, s, vh = np.linalg.svd(B)
    v = vh.T
    Q = v[:, -1]
    
    Q = Q.T / Q[-1] # <-- This scaling was highly recommended by Andreas <3
    
    return Q
    

From the book

In [ ]:
q1 = np.array([[300, 160]]).T

q2 = np.array([[300, 640]]).T

P1 = np.array([[800, 0, 300, 0], 
               [0, 800, 400, -2400],
               [0, 0, 1, 0]])

P2 = np.array([[800, 0, 300, 0],
               [0, 800, 400, 2400],
               [0, 0, 1, 0]])


In [ ]:
Q = triangulate(q_thicc = [q1, q2], P_thicc = [P1, P2])

print(Q)

[[300]
 [160]]
[[300]
 [640]]
[ 0.  0. 10.  1.]
